In [1]:
# !py -3 -m pip install pandas
# !py -3 -m pip install teradatasql
# !py -3 -m pip install teradatasqlalchemy

In [2]:
import pandas as pd
# import teradatasql
import sqlalchemy 

In [3]:
user_name = 'KHU9683'
pword = 'Zen18@dme'
host_name='PARAEDWPROD.DW.MEDCITY.NET'

In [4]:
td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=KRB5')

In [5]:
#path of file with table name and database name in csv file 
input_path = "InputFiles\\tables_get_rec_counts.csv"
output_path = "OutputFiles\\tables_rec_counts.csv"

In [6]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [7]:
#get the DDLs for the tables from teradata 
def get_rec_counts_table_updates():
    # with teradatasql.connect(host=host_name, user=user_name, password=pword, logmech="LDAP") as connect:
        df = pd.read_csv(input_path, index_col=None)        
        table_rec_counts = []
        for index, row in df.iterrows():
            try:
                valid_to_date_present = False

                database_name = str(row['Database']).strip().lower()
                table_name = str(row['Table']).strip().lower()

                query = "Select LOWER(columnname) AS columnname, UPPER(columntype) AS columntype "\
                    "From dbc.columnsV WHERE UPPER(DatabaseName) = '{}' AND UPPER(TableName) = '{}'"\
                    ";".format(database_name, table_name)
                results_df = pd.read_sql(query, td_engine)

                valid_to_date_present = 'valid_to_date' in results_df['columnname'].unique()
                if 'dw_last_update_date_time' in results_df['columnname'].unique():
                    dw_last_update_date_time_present = True
                    dw_last_update_field = 'dw_last_update_date_time'
                elif 'dw_last_update_date_tim' in results_df['columnname'].unique():
                    dw_last_update_date_time_present = True
                    dw_last_update_field = 'dw_last_update_date_tim'
                elif 'dw_last_update_time' in results_df['columnname'].unique():
                    dw_last_update_date_time_present = True
                    dw_last_update_field = 'dw_last_update_time'
                elif 'sk_generated_date_time' in results_df['columnname'].unique():
                    dw_last_update_date_time_present = True
                    dw_last_update_field = 'sk_generated_date_time'
                else:
                    dw_last_update_date_time_present = False
                    dw_last_update_field = ''

                dw_last_update_date = ""
                database_name_search = database_name if (database_name.endswith("_staging")) else (database_name + "_base_views")
                if (dw_last_update_date_time_present):
                    query = "Select Max({}) as {} FROM {}.{};".format(dw_last_update_field, dw_last_update_field, database_name_search, table_name)
                    results_df_last_update = pd.read_sql(query, td_engine)
                    dw_last_update_date = results_df_last_update[dw_last_update_field][0]

                rec_count = 0
                # if valid_to_date_present and not(database_name.endswith("_staging")):
                #     query = "Select Count(*) as rec_count from {}.{} Where CAST({} AS DATE) = SUBSTR('{}', 1, 10) "\
                #         "AND CAST(VALID_TO_DATE AS DATE) = '9999-12-31'"\
                #             ";".format(database_name_search, table_name, dw_last_update_field, dw_last_update_date)
                #     results_df_rec_count = pd.read_sql(query, td_engine)
                #     rec_count = results_df_rec_count['rec_count'][0]
                #     table_rec_counts.append((",").join([database_name, table_name + " Inserts", str(rec_count), str(dw_last_update_date)]))
                #     query = "Select Count(*) as rec_count from {}.{} Where CAST({} AS DATE) = SUBSTR('{}', 1, 10) "\
                #         "AND CAST(VALID_TO_DATE AS DATE) <> '9999-12-31'"\
                #             ";".format(database_name_search, table_name, dw_last_update_field, dw_last_update_date)
                #     results_df_rec_count = pd.read_sql(query, td_engine)
                #     rec_count = results_df_rec_count['rec_count'][0]
                #     table_rec_counts.append((",").join([database_name, table_name + " Updates", str(rec_count), str(dw_last_update_date)]))
                # elif dw_last_update_date_time_present:
                # if dw_last_update_date_time_present:
                #     query = "Select Count(*) as rec_count from {}.{} Where CAST({} AS DATE) = SUBSTR('{}', 1, 10)"\
                #     ";".format(database_name_search, table_name, dw_last_update_field, dw_last_update_date)
                #     results_df_rec_count = pd.read_sql(query, td_engine)
                #     rec_count = results_df_rec_count['rec_count'][0]
                #     table_rec_counts.append((",").join([database_name, table_name, str(rec_count), str(dw_last_update_date)]))
                # else:
                query = "Select SUM(CAST(1 AS BIGINT)) as rec_count from {}.{};".format(database_name_search, table_name)
                results_df_rec_count = pd.read_sql(query, td_engine)
                rec_count = results_df_rec_count['rec_count'][0]
                table_rec_counts.append((",").join([database_name, table_name, str(rec_count), str(dw_last_update_date)]))

            except Exception as e1:
                print(e1)
                print("Database : {}, Table : {}".format(database_name,table_name))
                pass

        write_file_local(output_path, table_rec_counts)
        print("Tables for Record Counts : ", len(df))

In [8]:
print("Begin of Processing")

get_rec_counts_table_updates()

print("End of Processing")

Begin of Processing
Tables for Record Counts :  22
End of Processing
